In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/digipodium/Datasets/main/kc_house_data.csv')
df.head()

In [ ]:
print(df.columns.tolist())

In [ ]:
df.drop(['id', 'date', 'zipcode'], axis=1, inplace=True)
df.head()

In [ ]:
df.corrwith(df['price']).plot.bar(figsize=(10,5), title='Correlation with price')

In [ ]:
X = df[['bathrooms','sqft_living']]
y  = df['price']    


In [ ]:
import xgboost as xgb

In [ ]:
data_matrix = xgb.DMatrix(X,y)

In [ ]:
params = {
    'learning_rate' : .01,
    'max_depth' : 15,
    'alpha' : 10,
}

results = xgb.cv(
    params,
    data_matrix,
    num_boost_round=500,
    early_stopping_rounds=5,
    metrics='rmse',
)
results['test-rmse-mean'].plot()

grid search

In [44]:
grid_params = [
    (max_depth, learning_rate, alpha)
    for max_depth in range(5,20,5)
    for learning_rate in [.01,.1,.5]
    for alpha in range(0,50,5)
]

min_rmse = float('inf')
best_params = None

for max_depth,learning_rate,alpha in grid_params:
    params = {
        'max_depth' : max_depth,
        'learning_rate' : learning_rate,
        'alpha' : alpha,
    }
    results = xgb.cv(
        params,
        data_matrix,
        num_boost_round=500,
        early_stopping_rounds=5,
        metrics='rmse',
    )
    current_rmse = results['test-rmse-mean'].min()
    if current_rmse < min_rmse:
        min_rmse = current_rmse
        best_params = (max_depth,learning_rate,alpha)
    print(f'Current best params: {best_params}')

Current best params: (5, 0.01, 0)
Current best params: (5, 0.01, 5)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (5, 0.01, 10)
Current best params: (